In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from scipy.optimize import curve_fit

In [4]:
filename = 'data/stokes_shift_05.csv'
data = pd.read_csv(filename, sep=',',header=1,nrows=151)
# Drop the 'Unnamed: 4' column as it contains no useful data
data = data.drop(columns=['Unnamed: 4'])
data.tail()

,Wavelength (nm),Intensity (a.u.),Wavelength (nm).1,Intensity (a.u.).1
146,NaN,NaN,545.969971,420.667816
147,NaN,NaN,547.010010,377.603302
148,NaN,NaN,548.049988,338.718384
149,NaN,NaN,548.950012,303.112579
150,NaN,NaN,550.000000,268.755066
